In [5]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime
from datetime import datetime
import requests

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


warnings.filterwarnings('ignore')

# Digital Printing

In [7]:
url_piutang = "https://docs.google.com/spreadsheets/d/1ZoQjTKYGjwDVYv6o1fhEU0V8JFTNyflu/export?format.xlsx"
url_penjualan = "https://docs.google.com/spreadsheets/d/1_Y-yKyCKaHKox5C3wcXMSV-88C_fsrl9/export?format.xlsx"

# baca data pelanggan
pl=pd.read_csv('/Users/dickyaryanto/Documents/Master Data/udrio/pelanggan/ipos/lcd_pelanggan_2023.csv', error_bad_lines=False, delimiter=';')

# baca data piutang
try:
    os.remove('piutang.xlsx')
except:
    None

output_filename = 'piutang.xlsx'

# get the data from spreadsheet
response = requests.get(url_piutang)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

pt = pd.read_excel('piutang.xlsx')
pt.reset_index(drop=True, inplace=True)

# transforming data piutang
pt['tanggal']=pd.to_datetime(pt['tanggal'])
pt['bayar']=pt['bayar'].fillna(0)
pt['sisa']=pt.kredit - pt.bayar
pt=pt[pt.sisa!=0]
pt['hari_jt']=pd.to_datetime(datetime.now()) - pt.tanggal
pt['hari_jt']=pt['hari_jt'].apply(lambda x: int(str(x).split()[0]))
pt=pt[['transaksi','tanggal','hari_jt','sisa']]
pt['transaksi']=pt['transaksi'].apply(lambda x: str(x))

# baca data penjualan
try:
    os.remove('penjualan.xlsx')
except:
    None

output_filename = 'penjualan.xlsx'

# get the data from spreadsheet
response = requests.get(url_penjualan)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

df2 = pd.read_excel('penjualan.xlsx')

# df=pd.concat([df1, df2], axis=0)
df = df2.copy()

df.reset_index(drop=True, inplace=True)
df['transaksi']=df['transaksi'].apply(lambda x: str(x))

# transforming data penjualan
df['tanggal']=pd.to_datetime(df['tanggal'])
df=df[df.transaksi.isin(pt.transaksi.unique().tolist())]
df['tunai']=df.tunai.fillna(0)
df['debit']=df.tunai.fillna(0)
df['bulan']=df.tanggal.dt.month
df['tahun']=df.tanggal.dt.year
df['date']=df.tanggal.dt.month.apply(lambda x: str(x) + '/') + df.tanggal.dt.year.apply(lambda x: str(x))


df['ket']=df['ket'].fillna('unknown')
df['cs']=df['cs'].fillna('unknown')

# transforming data penjualan hanya terhadap transaksi saja
_=[]
for i in df.transaksi.unique():
    n=df[df.transaksi == i]
    tunai=n.tunai.unique().tolist()[0]
    m=n.groupby(['transaksi','ket','customer','user','cs'])[['total']].sum().reset_index()
    debit=m.total.unique().tolist()[0] -  n.kredit.unique().tolist()[0]
    
    m['tunai']=tunai
    m['debet']=debit
    _.append(m)

# rekap df hanya dari transaksi saja
rekap_df=pd.concat(_)

def get_dp(tunai, debet):
    if tunai != 0:
        msg=tunai
    else:
        msg=debet
    return msg

rekap_df['bayar_awal'] = rekap_df.apply(lambda row: get_dp(row['tunai'], row['debet']), axis=1)
rekap_df=rekap_df.rename(columns={'user':'cs','cs':'operator'})
rekap_df['cs']=rekap_df['cs'].apply(lambda x: str(x).split()[0].split('(')[0].split('MLM')[0].split('_')[0])

#  merge (rekap_df, piutang)
eda=pd.merge(
    rekap_df[['transaksi','customer','cs','operator','total','tunai','debet','bayar_awal','ket']],
    pt,
    on='transaksi'
)[['transaksi','tanggal','hari_jt','customer','cs','operator','total','tunai','debet','bayar_awal','sisa','ket']]
eda=eda.drop_duplicates()
eda=eda[eda.hari_jt>3]
eda['cs']=eda.cs.apply(lambda x: ''.join(str(x).split()))

# merge (rekap_df, piutang), (pelanggan)
eda=pd.merge(
    eda,
    pl[['kode_customer','customer']],
    on='customer'
)[['transaksi','tanggal','hari_jt','kode_customer','customer','cs','operator','total','tunai','debet','bayar_awal','sisa','ket']]

# filter hanya customer yang tidak dikenali
eda['kode_customer']=eda.kode_customer.apply(lambda x: str(x).split('.')[0])
# eda=eda[
#     (eda.customer=='UMUM') |
#     (eda.customer=='UMUM KREDIT') |
#     (eda.customer=='CASH/TUNAI') |
#     (eda.customer=='ONLINE_WA_PRIBADI') | 
#     (eda.customer=='DIVISI_ONLINE_WA') |
#     (eda.kode_customer=='AB')
# ]

# eda = [eda.customer.isin(['UMUM','UMUM KREDIT','CASH/TUNAI','ONLINE_WA_PRIBADI','DIVISI_ONLINE_WA',''])]


In [11]:
eda

,transaksi,tanggal,hari_jt,kode_customer,customer,cs,operator,total,tunai,debet,bayar_awal,sisa,ket
0,402530,2024-01-03 09:09:26,194,PL0003,"KARYA PORE, UD",SORAYA,LUKMAN,50000,0,0,0,50000.0,KP1- 28749
1,402545,2024-01-03 09:54:08,194,PL0003,"KARYA PORE, UD",SORAYA,LUKMAN,100000,0,0,0,100000.0,KP2- 28751
2,402572,2024-01-03 11:00:53,194,PL0003,"KARYA PORE, UD",SORAYA,RIA,170000,0,0,0,170000.0,KP- KN STOFAN/AMBIL JAM 3 SORE
3,402576,2024-01-03 11:05:10,194,PL0003,"KARYA PORE, UD",SORAYA,LUKMAN,168000,0,0,0,168000.0,KP3- 28756 28755 28757
4,402653,2024-01-03 15:10:10,194,PL0003,"KARYA PORE, UD",SORAYA,ALEX,207500,0,0,0,207500.0,KP4- 28758 28766 28769 28739 28757 28763 28768
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1854,424525,2024-06-19 11:29:55,26,PL0670,HOTEL BEST WESTERN ( COCO ),ULAN,IKRAR,45000,0,0,0,45000.0,SPANDUK BEST WESTERN. DIAMBIL JAM 2. +62 821-9...
1855,426149,2024-07-01 11:10:43,14,PL0069,JEMY MAGALINE,SORAYA,DENY,67500,0,0,0,67500.0,BUKU TULIS GANTI COVER BUKU
1856,427109,2024-07-06 14:06:49,9,AB,ILMANSYAHRIZQI (CHALID KANTAO),NURUL,NURUL,57500,0,0,0,1800.0,ilman 2 ( stiker susu teras trift singa + glos...
1857,427501,2024-07-09 13:23:18,6,PL0817,NAYLA PANCAKE,NICIA,NICIA,11500,0,0,0,11500.0,STIKER AKSA


In [ ]:
eda=eda.rename(columns={'bayar_awal':'dp'})

eda=eda.sort_values(['cs','customer','hari_jt'], ascending=True)
eda=eda.dropna(axis=1)
eda=eda.drop_duplicates()
eda['transaksi']=eda['transaksi'].apply(lambda x: str(x).split('-')[0])
eda=eda.groupby(['cs','customer','transaksi','tanggal','hari_jt','ket'])[['total','dp','sisa']].sum()
eda['percent_dp']=(eda.dp/eda.total)
eda['percent_sisa']=(eda.sisa/eda.total)
eda=eda.reset_index()
eda.columns=['CS','CUSTOMER','TRANSAKSI','TANGGAL','JATUH TEMPO','KETERANGAN','TOTAL','DP','SISA','PERCENT DP','PERCENT SISA']
eda['TANGGAL']=eda['TANGGAL'].dt.date
eda=eda[eda['JATUH TEMPO']<100]
eda=eda.groupby(['CS','CUSTOMER','TRANSAKSI','TANGGAL','JATUH TEMPO','KETERANGAN'])[['TOTAL','DP','SISA','PERCENT DP','PERCENT SISA']].sum()
eda = eda.reset_index()

# merge dengan data sebelumnya
data_before = os.listdir('./before/')
_ = []
for i in data_before:
    path = './before/'
    data = pd.read_excel(path + i)
    _.append(data)
eda_bf = pd.concat(_)
eda_bf = eda_bf.drop_duplicates()
eda_bf['TRANSAKSI'] = eda_bf['TRANSAKSI'].apply(lambda x: str(x))

# merge data before and now
eda = eda.merge(eda_bf[['TRANSAKSI','STATUS']], how='left', on='TRANSAKSI')
eda = eda.drop_duplicates()
eda.fillna('', inplace=True)
eda = eda.groupby(['CS','CUSTOMER','TRANSAKSI','TANGGAL','JATUH TEMPO','STATUS','KETERANGAN'])[['TOTAL','DP','SISA','PERCENT DP','PERCENT SISA']].sum()

new_number = len(data_before) + 1

from datetime import datetime
date_now = str(datetime.now())

year = date_now.split('-')[0][-2:]
month = str(int(date_now.split('-')[1]))
day = date_now.split()[0].split('-')[-1]
date_name = day + '.' + month + '.' + year

name = str(new_number) + '. ' + 'lcd_check_piutang ' + date_name + '.xlsx'
eda.to_excel('./before/' + name)

# KaryaPore Balancing

In [ ]:
# # penjualan tunai
# pj_tunai=pd.read_csv('/Users/dickyaryanto/Documents/Master Data/udrio/penjualan_rekap/kp_penjualan_rekap.csv', delimiter=';')
# pj_tunai=pj_tunai[['transaksi','tanggal','kode_customer','customer','user','cs','tunai']]
# pj_tunai['tunai']=pj_tunai['tunai'].fillna(0)
# pj_tunai['tanggal']=pd.to_datetime(pj_tunai['tanggal'])
# pj_tunai['hari']=pj_tunai.tanggal.dt.strftime('%d/%m/%y')
# pj_tunai=pj_tunai[['transaksi','tanggal','hari','kode_customer','customer','user','cs','tunai']]
# pj_tunai['tunai']=pj_tunai['tunai'].apply(lambda x: int(str(x).split('.')[0]))
# pj_tunai=pj_tunai[pj_tunai.tunai>0]
# pj_tunai=pj_tunai.groupby(['hari'])[['tunai']].sum().reset_index()

# # pembayaran piutang tunai
# pb_tunai=pd.read_csv('/Users/dickyaryanto/Documents/Master Data/udrio/pelunasan/kp_pelunasan.csv', delimiter=';')
# pb_tunai=pb_tunai[pb_tunai.cara_bayar=='Tunai']
# pb_tunai['tanggal_byr']=pd.to_datetime(pb_tunai['tanggal_byr'])
# pb_tunai['hari']=pb_tunai.tanggal_byr.dt.strftime('%d/%m/%y')
# pb_tunai=pb_tunai.groupby(['hari'])[['bayar']].sum().reset_index()

# # merge
# data_ipos=pd.merge(pj_tunai, pb_tunai, on='hari')
# data_ipos['date']=pd.to_datetime(data_ipos['hari'])
# data_ipos['day']=data_ipos.date.dt.day
# data_ipos['month']=data_ipos.date.dt.month
# data_ipos=data_ipos.sort_values(['month','day'], ascending=True)
# data_ipos = data_ipos[['hari','tunai','bayar']]

# data_ipos['total'] = data_ipos.tunai + data_ipos.bayar
# data_ipos['tgl'] = data_ipos['hari'].apply(lambda x: int(str(x).split('/')[0]))
# data_ipos['bulan'] = data_ipos['hari'].apply(lambda x: int(str(x).split('/')[1]))
# data_ipos = data_ipos.sort_values(['bulan','tgl'], ascending=True)[['hari','tunai','bayar','total']]

# # get the data from url spreadsheet data_karyawan
# url = "https://docs.google.com/spreadsheets/d/1eyx8ZRKb1bv2cYWqy26Ck3qq_s2hWIR_XVBRXu0SZXU/export?format=xlsx"

# try:
#     os.remove('tbl_mutasi.xlsx')
# except:
#     None
    
# output_filename = "tbl_mutasi.xlsx"

# # get the data from spreadsheet
# response = requests.get(url)
# if response.status_code == 200:
#     with open(output_filename, "wb") as f:
#         f.write(response.content)

#     # read data
#     data_mutasi=pd.read_excel('tbl_mutasi.xlsx')
#     data_mutasi = data_mutasi[['Tanggal Laporan','Total Uang Fisik']]
#     data_mutasi['Tanggal Laporan'] = data_mutasi['Tanggal Laporan'].dt.strftime("%d/%m/%y")

# # merge data_ipos dan data_mutasi
# balancing = pd.merge(data_ipos, data_mutasi, left_on='hari', right_on='Tanggal Laporan', how='left')
# balancing['Total Uang IPOS'] = balancing.tunai + balancing.bayar
# balancing=balancing[['hari','Total Uang IPOS','Total Uang Fisik']]
# balancing.fillna(0, inplace=True)
# balancing.to_excel('KP BALANCING.xlsx', index=False)

# data_ipos.to_excel('data_tunai_ipos.xlsx', index=False)